In [1]:
import pandas as pd
import numpy as np

In [5]:
import pandas as pd

# 1. Defina o nome do arquivo exato
arquivo_excel = "BASE DE DADOS PEDE 2024 - DATATHON.xlsx"

try:
    # Vamos primeiro carregar o arquivo para ver quais são os nomes exatos das abas
    print("Lendo o arquivo Excel...")
    xls = pd.ExcelFile(arquivo_excel)
    
    # Isso vai imprimir os nomes das abas (ex: ['PEDE 2022', 'PEDE 2023'...])
    print("Abas encontradas:", xls.sheet_names)

    # 2. Carregamento das Abas
    # O Pandas é inteligente: se passarmos o objeto 'xls', ele não precisa reabrir o arquivo 3 vezes
    # OBS: Se os nomes no print acima forem diferentes (ex: "PEDE_2022"), ajuste abaixo
    df_2022 = pd.read_excel(xls, sheet_name="PEDE2022")
    df_2023 = pd.read_excel(xls, sheet_name="PEDE2023")
    df_2024 = pd.read_excel(xls, sheet_name="PEDE2024")

    print("\nSucesso!")
    print(f"2022: {df_2022.shape[0]} linhas")
    print(f"2023: {df_2023.shape[0]} linhas")
    print(f"2024: {df_2024.shape[0]} linhas")

except FileNotFoundError:
    print(f"ERRO: O arquivo '{arquivo_excel}' não foi encontrado.")
    print("Verifique se o arquivo está NA MESMA PASTA onde você rodou o comando 'jupyter notebook' ou criou o arquivo .ipynb")
except ValueError as e:
    print(f"ERRO DE NOME DE ABA: {e}")
    print("Verifique se os nomes 'PEDE 2022' batem exatamente com o que apareceu em 'Abas encontradas'.")


Lendo o arquivo Excel...
Abas encontradas: ['PEDE2022', 'PEDE2023', 'PEDE2024']

Sucesso!
2022: 860 linhas
2023: 1014 linhas
2024: 1156 linhas


In [10]:
# 2. Funções Auxiliares de Limpeza
def limpar_pedra(x):
    """Transforma a classificação Pedra em números para o modelo entender."""
    mapa_pedra = {
        'Quartzo': 1, 
        'Ágata': 2, 
        'Ametista': 3, 
        'Topázio': 4
    }
    # Se não estiver no mapa (ex: NaN), retorna NaN ou um valor neutro
    return mapa_pedra.get(x, np.nan)

def limpar_inde(x):
    """Converte INDE para float, tratando erros de vírgula/string."""
    if isinstance(x, str):
        try:
            return float(x.replace(',', '.'))
        except:
            return np.nan
    return x



In [11]:
# 3. Preparação dos Dados de 2022 (Apenas para histórico)
# Vamos pegar apenas o RA e o INDE para calcular a evolução
df_22_clean = df_2022[['RA', 'INDE 22']].copy()
df_22_clean.columns = ['RA', 'INDE_2022']

# 4. Preparação das Features (X) - Baseado em 2023
# Selecionando colunas essenciais
colunas_2023 = [
    'RA', 'INDE 2023', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN', 
    'Defasagem', 'Pedra 2023'
]
df_23_clean = df_2023[colunas_2023].copy()

# Renomeando para facilitar
df_23_clean.columns = [
    'RA', 'INDE_2023', 'IAA_2023', 'IEG_2023', 'IPS_2023', 'IDA_2023', 
    'IPP_2023', 'IPV_2023', 'IAN_2023', 'Defasagem_2023', 'Pedra_2023'
]

# Aplicando limpezas em 2023
df_23_clean['Pedra_2023_Num'] = df_23_clean['Pedra_2023'].apply(limpar_pedra)

# Juntando com 2022 para criar feature de evolução
df_features = pd.merge(df_23_clean, df_22_clean, on='RA', how='left')

# Feature Engineering: Tendência do INDE (2023 - 2022)
# Se positivo, aluno melhorou. Se negativo, piorou.
df_features['Delta_INDE'] = df_features['INDE_2023'] - df_features['INDE_2022']

# Preenchendo nulos da diferença com 0 (assumindo estabilidade para quem não tem dados de 22)
df_features['Delta_INDE'] = df_features['Delta_INDE'].fillna(0)


# 5. Preparação do Target (Y) - Baseado em 2024
# Precisamos saber quem piorou a defasagem em 2024
colunas_2024 = ['RA', 'Defasagem']
df_24_clean = df_2024[colunas_2024].copy()
df_24_clean.columns = ['RA', 'Defasagem_2024']

# 6. Unindo Features (2023) e Target (2024)
# Usamos 'inner join' pois só podemos treinar com alunos que temos a resposta (2024)
df_final = pd.merge(df_features, df_24_clean, on='RA', how='inner')

# Criando a Variável Alvo (Target): RISCO
# Lógica: É RISCO (1) se a defasagem de 2024 for MENOR que a de 2023.
# Exemplo: Em 2023 era 0 (ok), em 2024 virou -1 (atrasou). -1 < 0 -> True (Risco)
# Nota: Defasagem negativa é ruim.
df_final['TARGET_RISCO'] = (df_final['Defasagem_2024'] < df_final['Defasagem_2023']).astype(int)

# Removendo colunas que não entram no treino (como RA e textos originais)
# Mantemos RA apenas se quisermos identificar depois, mas dropamos pro modelo
colunas_para_treino = [
    'INDE_2023', 'IAA_2023', 'IEG_2023', 'IPS_2023', 'IDA_2023', 
    'IPP_2023', 'IPV_2023', 'IAN_2023', 'Defasagem_2023', 
    'Pedra_2023_Num', 'Delta_INDE', 'TARGET_RISCO'
]

df_modelo = df_final[colunas_para_treino].dropna()

print(f"Tamanho do Dataset Final: {df_modelo.shape}")
print("\nDistribuição do Target (0 = Normal, 1 = Piorou):")
print(df_modelo['TARGET_RISCO'].value_counts())

# Visualizar as primeiras linhas
df_modelo.head()

Tamanho do Dataset Final: (537, 12)

Distribuição do Target (0 = Normal, 1 = Piorou):
TARGET_RISCO
0    443
1     94
Name: count, dtype: int64


,INDE_2023,IAA_2023,IEG_2023,IPS_2023,IDA_2023,IPP_2023,IPV_2023,IAN_2023,Defasagem_2023,Pedra_2023_Num,Delta_INDE,TARGET_RISCO
0,9.31095,9.5,10.0,8.13,9.6,8.4375,8.920,10.0,0,4.0,0.0,1
1,8.22120,8.5,9.1,8.14,8.9,7.5000,8.585,5.0,-1,4.0,0.0,1
2,5.92975,0.0,7.6,3.14,6.3,5.9375,6.260,10.0,0,1.0,0.0,1
3,8.69920,9.0,8.6,8.76,8.9,7.5000,8.415,10.0,0,4.0,0.0,1
4,8.63895,10.0,9.5,8.76,7.1,7.1875,8.670,10.0,0,4.0,0.0,1
